In [ ]:
# We need to install 'ipython_unittest' to run unittests in a Jupyter notebook
!pip install -q ipython_unittest

You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [ ]:
# Loading modules that we need
from pyspark.sql.dataframe import DataFrame
from collections import Counter
from pyspark.sql.functions import desc

In [ ]:
# A helper function to load a table (stored in Parquet format) from DBFS as a Spark DataFrame 
def load_df(table_name: "name of the table to load") -> DataFrame:
    return spark.read.parquet(table_name)

users_df = load_df("/user/hive/warehouse/users")
posts_df = load_df("/user/hive/warehouse/posts")

#### Subtask 1: implementing two functions
Implement these two functions:
1. 'compute_pearsons_r' that receives a DataFrame and two column names and returns the [Pearson correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) between values of two columns;
2. 'make_tag_graph' that in the input receives the DataFrame containing the records related to 'questions' and returns a DataFrame with two columns 'u' and 'v'; the record for row i from the resulting DataFrame is a tuple (u_i, v_i). u_i and v_j are distinct tags and have appeared together for a question.

Please note that you should implement the 'compute_pearsons_r' yourself, so you should not use the 'DataFrame.stat.corr' method. Nevertheless, you can use 'DataFrame.stat.corr' to verify the correctness of your implementation.

### Documentation
#### Pearson correlation coefficient
$$\rho_{X,Y}=\frac{\text{cov}(X,Y)}{\sigma_X \sigma_Y}$$

### Spark functions
[Aggregate Functions](https://spark.apache.org/docs/3.3.1/api/python/reference/pyspark.sql/functions.html#aggregate-functions)

In [ ]:
from pyspark.sql.functions import stddev, col, size, array_sort, explode, count, udf, filter
import pyspark.sql.functions as F
import itertools
from pyspark.sql.types import *

def compute_pearsons_r(df: DataFrame, col1: str, col2: str) -> float:
  """
  Arguments:
    df: The DataFrame where col1 and col2 exist and should be used for calculations
    col1: The name of column X
    col2: The name of column Y
  """
  cov = df.stat.cov(col1, col2)
  df_stats = df.select(stddev(col1).alias("std1"), stddev(col2).alias("std2")).collect()
  return cov / (df_stats[0]["std1"] * df_stats[0]["std2"])

def make_tag_graph(df: DataFrame) -> DataFrame:
  """
  Arguments:
    df: The DataFrame containing question data
    
  Given the input
  +----+--------------+
  | Id | Tags         |
  +----+--------------+
  | 1  | [a, b, c, d] |
  | 2  | [b, c]       |
  | 3  | [f]          |
  +----+--------------+
  
  Should return a DataFrame:
  +---+---+
  | u | v |
  +---+---+
  | a | b |
  | a | c |
  | a | d |
  | b | a |
  | b | c |
  | b | d |
  | c | a |
  | c | b |
  | c | d |
  | d | a |
  | d | b |
  | d | c |
  | b | c |
  | c | b |
  +---+---+
  """
  
  # Create a UDF to generate all permutations of a given iterator
  #
  # Docs:
  # - [UDF](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.functions.udf.html?highlight=udf#pyspark.sql.functions.udf)
  # - [itertools.permutations](https://docs.python.org/3/library/itertools.html#itertools.permutations)
  permutations_udf = udf(
    lambda tags: list(itertools.permutations(tags, 2)),
    ArrayType(StructType([StructField("u", StringType(), False), StructField("v", StringType(), False)]))
  )
  
  # Filter out rows with null-tags, generate permutations with permutations_udf, and explode the resulting list into
  # separate columns
  df = df.filter(df.Tags.isNotNull()).select(explode(permutations_udf("Tags")).alias("permutations"))
  
  # Split the resulting struct{u, v} into separate columns u and v
  df = df.withColumn("u", col("permutations.u")).withColumn("v", col("permutations.v"))
  return df.select(col("u"), col("v"))


In [ ]:
# Validate that we've done it correctly
tol = 1e-15
assert (actual := abs(compute_pearsons_r(posts_df, "ViewCount", "Score") - posts_df.stat.corr("ViewCount", "Score"))) < tol, f"computed value is not within tolerance level: {actual=}, {tol=}"

In [ ]:
# Imprting GraphFrames graph library; make sure you have GraphFrames installed on the cluster
from graphframes import *

#### Subtask 2: implementing three functions
Impelment these three functions:
1. 'get_nodes' that, given the result from execution of 'make_tag_graph', returns a DataFrame with one column named 'id' that includes the tags that have appeared in the tag graph;
2. 'get_edges' that, given the result from execution of 'make_tag_graph', returns a DataFrame with two columns 'src' and 'dst' where 'src' is the source node and 'dst' is the destination node.
3. 'compute_pagerank' that receives a GraphFrames graph object in the input and computes the PageRank for nodes in the graph and returns the result as a DataFrame with two columns named 'id' and 'pagerank'; the rows in the in the resulting DataFrame should be sorted by the values of 'pagerank' column.

Note that the term 'tag graph' in this context refers to the DataFrame reuturned by executing 'make_tag_graph'. Furthermore, 'src' and 'dst' are distinct, so 'src' != 'dst'.

In [ ]:
def get_nodes(df: DataFrame) -> DataFrame:
  """"
  Since the tag graph is a permutation of the tags for a question, i.e. for a question with tags [a, b], both (a, b) and (b, a) will be
  present in the tag graph. As such, this function simply returns all distinct nodes in the column "u".
  
  Arguments:
    - df: DataFrame of the tag graph
    
  Returns:
    A DataFrame containing all nodes in df
  """
  return df.select(col("u").alias("id")).distinct()

def get_edges(df: DataFrame) -> DataFrame:
  """
  Arguments:
    - df: DataFrame of the tag graph
  
  Returns:
    A DataFrame with two columns ["src", "dst"] with all the edges in df
  """
  return df.withColumnRenamed("u", "src").withColumnRenamed("v", "dst")

def compute_pagerank(graph: GraphFrame) -> DataFrame:
  """
  https://graphframes.github.io/graphframes/docs/_site/api/python/graphframes.html#graphframes.GraphFrame.pageRank
  
  Arguments:
    - graph: A GraphFrame
  
  Returns:
    A DataFrame with the columns ["id", "pagerank"], sorted descendingly on pagerank
  """
  result = graph.pageRank(resetProbability=0.15, maxIter=10)
  return result.vertices.sort("pagerank", ascending=False).select("id", "pagerank")

In [ ]:
# Loading 'ipython_unittest' so we can use '%%unittest_main' magic command
%load_ext ipython_unittest

The ipython_unittest extension is already loaded. To reload it, use:
  %reload_ext ipython_unittest


#### Subtask 3: validating the implementation by running the tests

Run the cell below and make sure that all the tests run successfully.

In [ ]:
%%unittest_main
class TestTask3(unittest.TestCase):
  
  error_threshold = 0.03
  
  def test_corr1(self):
    # Pearson correlation coefficient between 'user reputation' and 'upvotes' received by users
    result = compute_pearsons_r(users_df, "Reputation", "UpVotes")
    self.assertLessEqual(abs(result-0.5218138310114108), self.error_threshold)
    print(result)
  
  def test_corr2(self):
    # Pearson correlation coefficient between 'user reputation' and 'downvotes' received by users
    result = compute_pearsons_r(users_df, "Reputation", "DownVotes")
    self.assertLessEqual(abs(result-0.1473558141546844), self.error_threshold)
    print(result)

  def test_corr3(self):
    # Pearson correlation coefficient between 'question score' and the 'number of answers' it received
    result = compute_pearsons_r(posts_df[posts_df["PostTypeId"] == 1], "Score", "AnswerCount")
    self.assertLessEqual(abs(result-0.47855272641249674), self.error_threshold)
    print(result)
    
  def test_make_tag_graph(self):
    result = make_tag_graph(df=posts_df[posts_df["PostTypeId"] == 1])
    self.assertIsInstance(result, DataFrame)
    
    coulmn_names = Counter(map(str.lower, ['u', 'v']))
    self.assertCountEqual(coulmn_names, Counter(map(str.lower, result.columns)), "Missing column(s) or column name mismatch")
    
    display(result)
    
    self.assertEqual(result.count(), 225290)
    
  def test_get_nodes(self):
    result = make_tag_graph(df=posts_df[posts_df["PostTypeId"] == 1])
    n = get_nodes(result)
    self.assertEqual(n.count(), 638)
    n.show()

  def test_get_edges(self):
    result = make_tag_graph(df=posts_df[posts_df["PostTypeId"] == 1])
    e = get_edges(result)
    
    coulmn_names = Counter(map(str.lower, ['src', 'dst']))
    self.assertCountEqual(coulmn_names, Counter(map(str.lower, e.columns)), "Missing column(s) or column name mismatch")
    
    self.assertEqual(e.count(), 225290)
    e.show()
    
  def test_compute_pagerank(self):
    result = make_tag_graph(df=posts_df[posts_df["PostTypeId"] == 1])
    n = get_nodes(result)
    e = get_edges(result)
    g = GraphFrame(n, e)
    ranks = compute_pagerank(g)
    self.assertEqual(ranks.first()[0], 'machine-learning')
    ranks.show()

+-------------------+------------------+
|                 id|          pagerank|
+-------------------+------------------+
|   machine-learning| 54.96146016212457|
|             python| 33.15439972701473|
|      deep-learning|24.867938087896142|
|     neural-network| 21.98498425240573|
|     classification|15.810465920358599|
|              keras|14.526328898904834|
|                nlp|11.797272420802912|
|       scikit-learn|11.320829324363698|
|         tensorflow|11.048800422404002|
|        time-series| 7.953902433102104|
|         regression| 7.450745600233994|
|            dataset| 7.214516413496702|
|                cnn| 7.144795504422983|
|                  r| 7.080681830287731|
|        data-mining| 6.759825281971387|
|         clustering| 6.505265732004028|
|predictive-modeling| 6.150883062336875|
|               lstm|5.9484019952350105|
|         statistics| 5.810739951509132|
|             pandas| 5.777956770501134|
+-------------------+------------------+
only showing top

u,v
education,open-source
open-source,education
data-mining,definitions
definitions,data-mining
machine-learning,bigdata
machine-learning,libsvm
bigdata,machine-learning
bigdata,libsvm
libsvm,machine-learning
libsvm,bigdata


Success

.......
----------------------------------------------------------------------
Ran 7 tests in 48.549s

OK
Out[80]: <unittest.runner.TextTestResult run=7 errors=0 failures=0>

#### Subtask 4: answering to questions about Spark related concepts

Please write a short description for the terms below---one to two short paragraphs for each term. Don't copy-paste; instead, write your own understanding.

1. What do the terms 'User-Defined Functions (UDFs)', 'Data Locality', 'Bucketing', 'Distributed Filesystem' mean in the context of Spark?

Write your descriptions in the next cell.

#### User-Defined Functions (UDFs)

User-defined functions are a feature of **Spark SQL**, allowing users to register custom routines that can then be invoked in a query. UDFs can take **columns** as arguments – when executing the query, columns are passed field-by-field to the UDF.

#### Data Locality

Data locality refers to _where_ Spark processes data. Spark applications run on a cluster of nodes, with data distributed among them. This distributed nature makes it advantageous to perform data processing on the node that stores that data, to avoid the overhead of moving data over the network between nodes. Spark strives for data locality by always trying to process data as close as possible to its location, although it may process on a less-local node if the best node is unavailable (the conditions for when this is done is configurable: https://spark.apache.org/docs/latest/configuration.html#scheduling).

#### Bucketing

Bucketing, like Data Locality, also relates to the distribution of data in a Spark application. It is a technique to reduce **shuffling** (moving of data over the network between nodes in the cluster) during execution of a query. This can be an expensive operation, so Spark will avoid it if possible, though it may be required for operations such as joining DataFrames.

Bucketing works by declaring a number of buckets, i.e. the number of partitions for the bucketed data, and the name of a column to use to sort the data into buckets. A hash function is applied to values of the column to give a **bucket number**, and the data is then moved to that bucket. This entails an initial overhead when shuffling and sorting data into buckets, but avoids the later shuffling during query execution. Since these buckets can be reused across queries, it can give substantial performance benefits by avoiding shuffling for each query.

#### Distributed Filesystem

In a distributed filesystem, stored files are distributed across multiple different servers. There can be many reasons to use such a system:
- Supporting the storing of massive datasets that need the storage capacity of multiple machines
- Replicating data across machines to reduce the risk of losing it
- Locating data closer to users in different geographical regions

In the context of Spark, a distributed filesystem is typically used as a **data source**. Spark is compatible with several different distributed filesystems, such as the Hadoop Distributed File System (HDFS), HBase, Hive and Cassandra.